In [2]:
%load_ext autoreload
%autoreload 2
from ASD_Circuits import *

HGNC = pd.read_csv("/Users/jiayao/Work/Resources/protein-coding_gene.txt", delimiter="\t")
ENSID2Entrez = dict(zip(HGNC["ensembl_gene_id"].values, HGNC["entrez_id"].values))
GeneSymbol2Entrez = dict(zip(HGNC["symbol"].values, HGNC["entrez_id"].values))
Entrez2Symbol = dict(zip(HGNC["entrez_id"].values, HGNC["symbol"].values))
allen_mouse_genes = loadgenelist("../dat/allen-mouse-exp/allen-mouse-gene_entrez.txt")

/Users/jiayao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (38,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# ASD

In [3]:
DenovoWEST_TADA = pd.read_csv("../dat/genes/asd/DenovoWEST_TADA.txt", delimiter="\t")

In [6]:
print(DenovoWEST_TADA.columns.values)

['GeneID' 'HGNC' 'EntrezID' 'pLI' 'LOUEFbin' 'LOUEF' 'LoFOvsE' 'SynOvsE'
 'sHet' 'forecASD' 'Frisk' 'SFARICategory' 'SFARIScore' 'DDDCategory'
 'DDDAllelic' 'DDDMutCons' 'DDDOrgan' 'GeneSet' 'dnLGD' 'mutLGD'
 'log10BF_dnLGD' 'dnDmis' 'mutDmis' 'log10BF_dnDmis' 'log10BF_All'
 'PP_All' 'Qvalue' 'AllObserved' 'AllExpected' 'pAllEnrich' 'MisObserved'
 'MisExpected' 'pMisEnrich' 'MisEvents' 'MisDist' 'pMisCluster' 'pMisComb'
 'pDenovoWEST']


In [4]:
DenovoWEST_TADA.head(2)

,GeneID,HGNC,EntrezID,pLI,LOUEFbin,LOUEF,LoFOvsE,SynOvsE,sHet,forecASD,...,AllExpected,pAllEnrich,MisObserved,MisExpected,pMisEnrich,MisEvents,MisDist,pMisCluster,pMisComb,pDenovoWEST
0,ENSG00000136531,SCN2A,6326,0.999999992314383,0,1.2700e-01,6.0487e-02,1.0084e+00,0.257771076876808,1,...,0.7341930968145816,1.00E-14,6.759,0.5579805995344479,1.69E-16,21,1271.4,0.27,1.76E-15,1.76E-15
1,ENSG00000100888,CHD8,57680,0.999999999998095,0,8.2000e-02,3.8764e-02,9.4139e-01,0.155416372713663,1,...,0.8339649944288832,1.00E-14,4.194,0.5547438680353904,3.51E-09,14,679.5,1.49E-04,1.53E-11,1.00E-14


In [8]:
TotalN = 16877
TotalLGD = sum(DenovoWEST_TADA["dnLGD"].values)
ExpLGD = sum(DenovoWEST_TADA["mutLGD"].values) * 2 * TotalN
print(TotalLGD, ExpLGD, (TotalLGD - ExpLGD)/TotalLGD)

TotalDmis = sum(DenovoWEST_TADA["dnDmis"].values)
ExpDmis = sum(DenovoWEST_TADA["mutDmis"].values) * 2 * TotalN
print(TotalDmis, ExpDmis, (TotalDmis - ExpDmis)/TotalDmis)

2696 1732.863855985384 0.35724634421907125
3326 2556.112713640646 0.23147543185789363


In [3]:
spark_tada = pd.read_csv("/Users/jiayao/Work/ASD_Circuits/dat/genes/TADA.SPARK.ONLY.txt", delimiter="\t")
print(spark_tada.columns.values)
spark_tada.head(2)

['GeneID' 'HGNC' 'ExACpLI' 'gnomADpLI' 'LoFOvsE' 'sHet' 'SFARICategory'
 'SFARIScore' 'DDDCategory' 'DDDAllelic' 'DDDMutCons' 'DDDOrgan' 'dnLGD'
 'mutLGD' 'log10BF_dnLGD' 'dnDmis' 'mutDmis' 'log10BF_dnDmis'
 'log10BF_All' 'PP_All' 'Qvalue']


,GeneID,HGNC,ExACpLI,gnomADpLI,LoFOvsE,sHet,SFARICategory,SFARIScore,DDDCategory,DDDAllelic,...,DDDOrgan,dnLGD,mutLGD,log10BF_dnLGD,dnDmis,mutDmis,log10BF_dnDmis,log10BF_All,PP_All,Qvalue
0,ENSG00000101126,ADNP,0.9989040999292,1.0000e+00,2.5868e-02,0.135996662587073,"Rare Single Gene Mutation, Syndromic, Functional",1,confirmed,monoallelic,...,Brain/Cognition,12,0.000005,16.25,2,0.000005,1.36,17.62,1.0,9.990000e-16
1,ENSG00000136531,SCN2A,0.999999992314383,1.0000e+00,6.0487e-02,0.257771076876808,"Rare Single Gene Mutation, Syndromic",1,confirmed,monoallelic,...,Brain/Cognition,10,0.000010,12.33,7,0.000048,3.86,16.20,1.0,5.500000e-15


In [15]:
NTrios = 7015
for i, row in spark_tada.iterrows():
    row["LGD_exp"] = row["mutLGD"] * NTrios * 2
    row["LGD_RR"] = row["dnLGD"]/row["LGD_exp"]
    row["Dmis_exp"] = row["mutDmis"] * NTrios * 2
    row["Dmis_RR"] = row["dnDmis"]/row["Dmis_exp"]
    spark_tada.loc[i, "LGD_RR"] = max(1, row["LGD_RR"])
    spark_tada.loc[i, "Dmis_RR"] = max(1, row["Dmis_RR"])
    try:
        spark_tada.loc[i, "Entrez"] = int(GeneSymbol2Entrez[row["HGNC"]])
    except:
        spark_tada.loc[i, "Entrez"] = "."
spark_tada.to_csv("../dat/genes/spark_tada_RR.csv", index=False)

In [9]:
asc_mutable = pd.read_excel("../dat/genes/1-s2.0-S0092867419313984-mmc2.xlsx", sheet_name="Autosomal")
print(asc_mutable.columns.values)
asc_mutable.head(2)

['gene' 'hugoGene' 'hgnc_id' 'entrez_id' 'ensembl_gene_id'
 'refseq_accession' 'uniprot_ids' 'location' 'chr' 'start_hg38'
 'stop_hg38' 'strand' 'cDNA' 'exons' 'dn.ptv' 'dn.misa' 'dn.misb'
 'dbs.case.ptv' 'dbs.control.ptv' 'sweden.case.ptv' 'sweden.control.ptv'
 'case.ptv' 'control.ptv' 'mut.ptv' 'mut.misa' 'mut.misb' 'pLI'
 'gamma_dn.ptv' 'gamma_dn.misa' 'gamma_dn.misb' 'BF_dn.ptv' 'BF_dn.misa'
 'BF_dn.misb' 'gamma_cc_ptv' 'rho.ptv' 'BF_cc.ptv' 'qval_dnccPTV'
 'ddidDN.ptv' 'ddidDN.misb' 'ddidDN.misa' 'asd_rate_dn' 'asd_dn' 'ddid_dn'
 'ddid_rate_dn' 'Transmitted_PTV_proband' 'Untransmitted_PTV_proband'
 'ASC33_2014' 'SSC27_2014' 'ASC65_2015' 'ASC102_2018' 'ASD:DDID'
 'ASD_vs_DDID' 'ASD53' 'DDID49' 'Function']


,gene,hugoGene,hgnc_id,entrez_id,ensembl_gene_id,refseq_accession,uniprot_ids,location,chr,start_hg38,...,Untransmitted_PTV_proband,ASC33_2014,SSC27_2014,ASC65_2015,ASC102_2018,ASD:DDID,ASD_vs_DDID,ASD53,DDID49,Function
0,CHD8,CHD8,HGNC:20153,57680,ENSG00000100888,NM_020920,Q9HCK8,14q11.2,chr14,21385194,...,1,1,1,1,1,3.07851,ASD,1,0,Gene expression regulation
1,SCN2A,SCN2A,HGNC:10588,6326,ENSG00000136531,NM_021007,Q99250,2q24.3,chr2,165194993,...,0,1,1,1,1,0.691315,DDID,0,1,Neuronal communication


In [17]:
NTrios = 6430
for i, row in asc_mutable.iterrows():
    row["LGD_exp"] = row["mut.ptv"] * NTrios * 2
    row["LGD_RR"] = row["dn.ptv"]/row["LGD_exp"]
    try:
        row["misa_exp"] = row["mut.misa"] * NTrios * 2
        row["misa_RR"] = row["dn.misa"]/row["misa_exp"]
    except:
        row["misa_RR"] = 1
    try:
        row["misb_exp"] = row["mut.misb"] * NTrios * 2
        row["misb_RR"] = row["dn.misb"]/row["misb_exp"]
    except:
        row["misb_RR"] = 1
    asc_mutable.loc[i, "LGD_RR"] = max(1, row["LGD_RR"])
    asc_mutable.loc[i, "misa_RR"] = max(1, row["misa_RR"])
    asc_mutable.loc[i, "misb_RR"] = max(1, row["misb_RR"])
asc_mutable.to_csv("../dat/genes/asc_tada_RR.csv", index=False)

In [ ]:
spark_tada.head(10)

In [23]:
LGDs = spark_tada[spark_tada["dnLGD"]>=1]

In [26]:
sum(spark_tada["dnLGD"].values)

1092

In [28]:
sum(spark_tada["dnLGD"].values[:200])

336

In [24]:
LGDs

(908, 21)

In [ ]:
spark_tada.head(5)

In [4]:
print(spark_tada[spark_tada["Qvalue"]<0.5].shape)

(303, 21)


In [6]:
WriteGeneList([GeneSymbol2Entrez[x] for x in spark_tada[spark_tada["Qvalue"]<0.5]["HGNC"].values if x in GeneSymbol2Entrez],
             "/Users/jiayao/Work/ASD_Circuits/dat/genes/asd.spark.tada.300.txt")

In [15]:
ssc_spark_tal = pd.read_csv("../dat/genes/spark-simplex.csv")
for i, row in ssc_spark_tal.iterrows():
    try:
        ssc_spark_tal.loc[i, "SYMBOL"] = Entrez2Symbol[int(row["GENE"])]
    except:
        ssc_spark_tal.loc[i, "SYMBOL"] = "NA"
ssc_spark_tal.head(10)

,GENE,MUT.TYPE,A.D,nA.D,OBS.SYN,EXP,OBS,R.A,R.nA,A.DR,FDR,SYMBOL
0,5310.0,rec.lgd,0.90,0.10,246.0,9.905,0.0,0.371391,0.000050,0.999985,0.000015,PKD1
1,5310.0,rec.mis,0.35,0.65,246.0,9.905,0.0,0.371391,0.000050,0.999750,0.000250,PKD1
2,8295.0,rec.mis,0.35,0.65,208.0,8.738,0.0,0.417363,0.000160,0.999287,0.000713,TRRAP
3,2033.0,rec.lgd,0.90,0.10,117.0,5.587,0.0,0.571952,0.003746,0.999273,0.000727,EP300
4,4297.0,rec.lgd,0.90,0.10,116.0,5.554,0.0,0.573843,0.003872,0.999251,0.000749,KMT2A
5,58499.0,rec.lgd,0.90,0.10,113.0,5.461,0.0,0.579204,0.004249,0.999186,0.000814,ZNF462
6,22941.0,rec.lgd,0.90,0.10,106.0,5.255,0.0,0.591260,0.005221,0.999020,0.000980,SHANK2
7,29123.0,lgd,0.43,0.57,179.0,7.824,0.0,0.457307,0.000400,0.998842,0.001158,ANKRD11
8,23499.0,lgd,0.43,0.57,176.0,7.725,0.0,0.461857,0.000442,0.998734,0.001266,MACF1
9,29123.0,rec.mis,0.35,0.65,179.0,7.824,0.0,0.457307,0.000400,0.998378,0.001622,ANKRD11


In [16]:
ssc_spark_tal.to_csv("../dat/genes/spark_wiglermodel.csv", index=False)

# SCZ

In [3]:
# Jan 2020 SCZ paper
Howrigan_2020_gene_df = pd.read_excel("/Users/jiayao/Work/ASD_Circuits/dat/genes/scz/raw/SCZ_TW_Recrrent_PTV_Missense.xlsx")

Howrigan_2020_CB = Howrigan_2020_gene_df[Howrigan_2020_gene_df["Cohort"]=="CB"]
Howrigan_2020_TW = Howrigan_2020_gene_df[Howrigan_2020_gene_df["Cohort"]=="TW"]

In [27]:
# Recurrent PTV
Howrigan_2020_CB_LGD = Howrigan_2020_CB[Howrigan_2020_CB["PTV count"]>1]
Howrigan_2020_TW_LGD = Howrigan_2020_TW[Howrigan_2020_TW["PTV count"]>1]
Howrigan_2020_LGD = list(Howrigan_2020_CB_LGD["gene"].values) + list(Howrigan_2020_TW_LGD["gene"].values)
Howrigan_2020_LGD = list(set([GeneSymbol2Entrez[x] for x in Howrigan_2020_LGD if x in GeneSymbol2Entrez]))
WriteGeneList(Howrigan_2020_LGD, "/Users/jiayao/Work/ASD_Circuits/dat/genes/scz/Howrigan_2020_rec_LGD.txt")

In [29]:
# PTV with at least one missense
Howrigan_2020_CB_LGD = Howrigan_2020_CB[Howrigan_2020_CB["PTV count"]>=1]
Howrigan_2020_TW_LGD = Howrigan_2020_TW[Howrigan_2020_TW["PTV count"]>=1]
Howrigan_2020_LGD = list(Howrigan_2020_CB_LGD["gene"].values) + list(Howrigan_2020_TW_LGD["gene"].values)
Howrigan_2020_LGD = list(set([GeneSymbol2Entrez[x] for x in Howrigan_2020_LGD if x in GeneSymbol2Entrez]))
WriteGeneList(Howrigan_2020_LGD, "/Users/jiayao/Work/ASD_Circuits/dat/genes/scz/Howrigan_2020_LGD.txt")

In [35]:
# Rec genes
Howrigan_2020 = list(Howrigan_2020_CB["gene"].values) + list(Howrigan_2020_TW["gene"].values)
Howrigan_2020 = list(set([GeneSymbol2Entrez[x] for x in Howrigan_2020 if x in GeneSymbol2Entrez]))
WriteGeneList(Howrigan_2020, "/Users/jiayao/Work/ASD_Circuits/dat/genes/scz/Howrigan_2020_rec.txt")

In [ ]:
# Owen 2020 SCZ LGDs
Genes = [""]

# Simulate Healthy Indvs

In [116]:
Nindv = 16877
BGMR = pd.read_csv("/Users/jiayao/Work/Resources/MutationRate_20170711_rate.txt", delimiter="\t")
#BGMR.index=BGMR["GeneName"].values
print(BGMR.shape)
BGMR["Entrez"] = [int(GeneSymbol2Entrez.get(x, -1)) for x in BGMR["GeneName"].values]
#BGMR = BGMR[BGMR["Entrez"].isin(allen_mouse_genes)]
BGMR.index=BGMR["Entrez"].values
print(BGMR.shape)

(18954, 64)
(18954, 65)


/Users/jiayao/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [123]:
N_simulations = 10
N_indv = 1000

In [124]:
DAT = []
for i in range(N_simulations):
    LGDs1 = np.random.rand(N_indv, BGMR.shape[0])
    LGDs2 = np.random.rand(N_indv, BGMR.shape[0])
    Dmis1 = np.random.rand(N_indv, BGMR.shape[0])
    Dmis2 = np.random.rand(N_indv, BGMR.shape[0])
    LGD_idx = np.array([])
    Dmis_idx = np.array([])
    for j in range(N_indv):
        j_lgd_1 = np.where(BGMR["p_LGD"].values > LGDs1[j, :])
        j_lgd_2 = np.where(BGMR["p_LGD"].values > LGDs2[j, :])
        j_dmis_1 = np.where(BGMR["prevel_0.5"].values > Dmis1[j, :])
        j_dmis_2 = np.where(BGMR["prevel_0.5"].values > Dmis2[j, :])
        LGD_idx = np.append(LGD_idx, j_lgd_1)
        LGD_idx = np.append(LGD_idx, j_lgd_2)
        Dmis_idx = np.append(Dmis_idx, j_dmis_1)
        Dmis_idx = np.append(Dmis_idx, j_dmis_2)
        k,v = np.unique([BGMR.index[k] for k in LGD_idx], return_counts=True)
        Entrez_LGD = dict(zip(k, v))
        k,v = np.unique([BGMR.index[k] for k in Dmis_idx], return_counts=True)
        Entrez_Dmis = dict(zip(k, v))
        dat = {"LGD": Entrez_LGD, "Dmis": Entrez_Dmis}
    DAT.append(dat)

In [132]:
for dat in DAT:
    nlgd = len(dat["LGD"])
    print(nlgd / 100)

0.88
0.74
0.9
0.9
0.76
0.91
0.84
0.96
0.75
0.97


In [94]:
# Clean up data
# Collect All Keys:
keys = []
for dat in DAT:
    keys.extend(list(dat["LGD"].keys()) + list(dat["Dmis"].keys()))
keys = list(set(keys))

In [101]:
len(keys)

110

In [96]:
dat_df = []
for dat in DAT:
    row_df = []
    for k in keys:
        lgd = dat["LGD"].get(k, 0)
        dmis = dat["Dmis"].get(k, 0)
        row_df.append("%d,%d"%(lgd,dmis))
    dat_df.append(row_df)

In [108]:
print(len(dat_df), len(dat_df[0]))

10 110


In [109]:
df = pd.DataFrame(dat_df, columns=keys, index=list(range(N_simulations)))

In [113]:
df["index"] = df.index
df = df.set_index('index')

In [115]:
df.to_csv("dat/meta_denovo_simulations.csv")

In [118]:
N_simulations = 10
N_indv = 100

In [78]:
LGDs = np.random.rand(N_indv, BGMR.shape[0])
Dmis = np.random.rand(N_indv, BGMR.shape[0])
LGD_idx = np.array([])
Dmis_idx = np.array([])
for j in range(N_indv):
    j_lgd = np.where(BGMR["p_LGD"].values > LGDs[j, :])
    j_dmis = np.where(BGMR["prevel_0.5"].values > Dmis[j, :])
    LGD_idx = np.append(LGD_idx, j_lgd)
    Dmis_idx = np.append(Dmis_idx, j_dmis)
    k,v = np.unique([BGMR.index[k] for k in LGD_idx], return_counts=True)
    Entrez_LGD = dict(zip(k, v))
    k,v = np.unique([BGMR.index[k] for k in Dmis_idx], return_counts=True)
    Entrez_Dmis = dict(zip(k, v))

In [87]:
dat = {"LGD": Entrez_LGD, "Dmis": Entrez_Dmis}

In [133]:
## Load
df = pd.read_csv("dat/meta_denovo_simulations.csv", index_col="index")
df.head(2)

,339977,2058,6157,10257,24,4125,84000,387104,55343,4145,...,4037,389072,4054,79837,51168,79841,10223,157680,51185,4086
index,,,,,,,,,,,,,,,,,,,,,
0,"0,0","0,0","0,0","0,1","0,0","0,0","0,0","0,0","0,0","1,0",...,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
1,"0,0","0,0","1,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0",...,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"


In [134]:
for i, row in df.iterrows():
    nlgd = [int(x.split(",")[0]) for x in row.values]
    print(sum(nlgd)/100)

0.03
0.1
0.04
0.11
0.12
0.05
0.08
0.06
0.14
0.07
0.09
0.09
0.07
0.07
0.09
0.03
0.09
0.04
0.1
0.11


# Cont